In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.io.fits as fits
from scipy.optimize import leastsq
from numpy import exp
from scipy import ndimage
from func import *
from astropy.table import Table, hstack
import os
import glob
from astropy.table import Table, vstack, join
import pandas as pd
from shutil import copyfile

In [2]:
def scale_factor_spectrum(z,data,error,wave):
    k = 1+z
    select1 = (wave>6750*k) & (wave<6800*k)
    cont_dat = data[select1]
    cont_err = error[select1]
    error_maincube = np.std(cont_dat)
    error_errcube = np.mean(cont_err)
    fact = error_maincube/error_errcube
    return fact

In [3]:
def aperture_spec_fixkin_fit(int_spectrum,int_error,mini_wave,fixed_param,broad2=False,MC_loops=0):
    if broad2:
        full_gauss_fixkin = full_gauss2_fixkin
    else:
        full_gauss_fixkin = full_gauss1_fixkin
    (spectrum,error) = (int_spectrum,int_error)   
    popt_full_fit,pcov_full_fit = leastsq(full_gauss_fixkin,x0=[0.1,0.1,5,5,0.01,0.01,0.01,0.01,-0.7,0.001],args=(mini_wave,spectrum,error,fixed_param),maxfev = 10000000)       
    fitted=(full_gauss_fixkin(popt_full_fit,mini_wave,spectrum,error,fixed_param))*(error)+spectrum 
    residual = spectrum - fitted
    
    spec_parameters_MC = np.zeros((len(popt_full_fit),MC_loops))
    for l in range(MC_loops):
        iteration_data = np.random.normal(spectrum,error)   
        popt_spec_MC,pcov_spec_MC = leastsq(full_gauss_fixkin,x0=[0.1,0.1,5,5,0.01,0.01,0.01,0.01,-0.7,0.001],args=(mini_wave,iteration_data,error,fixed_param),maxfev = 10000000)
        spec_parameters_MC[:,l]=popt_spec_MC
    spec_parameters_err = np.std(spec_parameters_MC,1)
    return popt_full_fit,spec_parameters_err,fitted,residual

def central_table(obj,output_par,output_par_err,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    column_names={'amp_Hb':0,'amp_OIII5007':1,'amp_OIII5007_br':2,'amp_Hb_br':3,'amp_Hb1':4,'amp_Fe5018_1':5,'amp_Hb2':6,'amp_Fe5018_2':7,'m':8,'c':9}
    columns=[]
    for key in column_names.keys():
        columns.append(fits.Column(name=key,format='E',array=[output_par[column_names[key]]]))
        columns.append(fits.Column(name=key+'_err',format='E',array=[output_par_err[column_names[key]]]))
    coldefs = fits.ColDefs(columns)
    hdu = fits.BinTableHDU.from_columns(coldefs)
    hdu.writeto('%s/%s/%s_spectro_central_fit.fits'%(destination_path_cube,obj,obj),overwrite=True)

def central_table_with_source(obj,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    t1 = Table.read('%s/%s/source_%s.fits'%(destination_path_cube,obj,obj),format='fits')
    t2 = Table.read('%s/%s/%s_spectro_central_fit.fits'%(destination_path_cube,obj,obj),format='fits')
    new = hstack([t1, t2])
    new.write('%s/%s/%s_fluxdensity3.fits'%(destination_path_cube,obj,obj),overwrite=True)
    


In [4]:
def spectroastrometric_script(obj,z,broad2,MC_loops=100,destination_path_cube="/home/rickeythecat/Downloads/Outflow_paper1/MUSE"):
    k = 1+z
    print ('%s'%(obj))
    fixed_param = fixed_parameters(obj)
    (mini_cube,mini_err,mini_wave,mini_header) = loadCube('%s/%s/%s.3_arcsec_minicube.fits'%(destination_path_cube,obj,obj))
    (int_spectrum,int_error) = int_spec(mini_cube,mini_err)
    (output_par,output_par_err,fitted,residual) = aperture_spec_fixkin_fit(int_spectrum,int_error,mini_wave,fixed_param,broad2,MC_loops=100)
    print (output_par,output_par_err)
    central_table(obj,output_par,output_par_err)
    central_table_with_source(obj)

In [5]:
z = {"HE0021-1810":0.05352}

objs = z.keys()

broad2= {'HE0021-1810':False}


for obj in objs:
    spectroastrometric_script(obj,z[obj],broad2[obj])   

HE0021-1810
[1.09481342e+00 1.60328630e+01 1.16458153e+01 5.06441401e-01
 1.20497551e+00 1.00000000e-02 1.95853602e+00 1.00000000e-02
 1.31857865e+00 9.88049658e-01] [1.33011876e-01 2.27506779e-01 1.20136166e-01 7.83916650e-02
 7.01647495e-02 1.73472348e-18 4.59873361e-02 1.73472348e-18
 2.59418998e-01 1.37935233e+00]


In [6]:
z = {"HE0021-1810":0.05352,"HE0021-1819":0.053197,"HE0040-1105":0.041692,"HE0108-4743":0.02392,"HE0114-0015":0.04560
    ,"HE0119-0118":0.054341,"HE0212-0059":0.026385,"HE0224-2834":0.059800,"HE0227-0913":0.016451,"HE0232-0900":0.043143
    ,"HE0253-1641":0.031588,"HE0345+0056":0.031,"HE0351+0240":0.036,"HE0412-0803":0.038160,"HE0429-0247":0.042009
    ,"HE0433-1028":0.035550,"HE0853+0102":0.052,"HE0934+0119":0.050338,"HE1011-0403":0.058314,"HE1017-0305":0.049986
    ,"HE1029-1831":0.040261,"HE1107-0813":0.058,"HE1108-2813":0.024013,"HE1126-0407":0.061960,"HE1237-0504":0.009
    ,"HE1248-1356":0.01465,"HE1330-1013":0.022145,"HE1353-1917":0.035021,"HE1417-0909":0.044,"HE2128-0221":0.05248
    ,"HE2211-3903":0.039714,"HE2222-0026":0.059114,"HE2233+0124":0.056482,"HE2302-0857":0.046860}

objs = z.keys()

broad2= {'HE0021-1810':False,'HE0021-1819':False,'HE0040-1105':False,'HE0108-4743':True,'HE0114-0015':False
        ,'HE0119-0118':True,'HE0212-0059':True,'HE0224-2834':False,'HE0227-0913':True,'HE0232-0900':False
        ,'HE0253-1641':True,'HE0345+0056':True,'HE0351+0240':True,'HE0412-0803':False,'HE0429-0247':True
        ,'HE0433-1028':True,'HE0853+0102':True,'HE0934+0119':True,'HE1011-0403':True,'HE1017-0305':False
        ,'HE1029-1831':True,'HE1107-0813':True,'HE1108-2813':False,'HE1126-0407':True,'HE1237-0504':False
        ,'HE1248-1356':False,'HE1330-1013':True,'HE1353-1917':True,'HE1417-0909':False,'HE2128-0221':False
        ,'HE2211-3903':False,'HE2222-0026':True,'HE2233+0124':True,'HE2302-0857':True}


for obj in objs:
    spectroastrometric_script(obj,z[obj],broad2[obj])   

HE0021-1810
[1.09481342e+00 1.60328630e+01 1.16458153e+01 5.06441401e-01
 1.20497551e+00 1.00000000e-02 1.95853602e+00 1.00000000e-02
 1.31857865e+00 9.88049658e-01] [1.41607592e-01 2.38471347e-01 1.32631623e-01 9.50280395e-02
 7.37725596e-02 1.73472348e-18 4.43190739e-02 1.73472348e-18
 2.79476539e-01 1.48132644e+00]
HE0021-1819
[ 4.38984735e+00  4.49667043e+01  8.58627417e+00  1.29410281e+00
  1.31338865e+00  1.00000000e-02  1.98068137e-01  1.00000000e-02
 -8.53712386e-01  5.96448075e+00] [3.05718227e-02 6.28816107e-02 2.12145331e-02 1.76811288e-02
 6.29738049e-03 1.73472348e-18 5.68856077e-03 1.73472348e-18
 1.70700676e-02 8.83351088e-02]
HE0040-1105
[ 8.75089614e+00  9.02862914e+01  1.24805232e+01  2.00760014e+00
  7.42245330e+00  1.00000000e-02  9.60048314e-01  1.00000000e-02
  9.29586949e-01 -3.73525878e-01] [5.54058927e-02 1.02080679e-01 3.47490427e-02 3.55631322e-02
 1.41451406e-02 1.73472348e-18 9.57137437e-03 1.73472348e-18
 2.99811262e-02 1.52774885e-01]
HE0108-4743
[39.5276

[ 5.19300562e+00  3.98857328e+01  9.93355296e+00  3.13778287e+00
  5.32983331e+00  1.00000000e-02  6.70612887e-01  1.00000000e-02
 -3.70167089e-01  5.81273979e+00] [9.01355203e-02 9.09140446e-02 5.99900270e-02 7.34344096e-02
 1.54156148e-02 1.73472348e-18 8.07801713e-03 1.73472348e-18
 2.07845476e-02 1.08322463e-01]
HE2211-3903
[8.06417674e+00 4.54425145e+01 1.83097587e+00 2.41203630e-01
 4.52077446e+00 1.00000000e-02 7.13315509e-01 1.00000000e-02
 6.27408006e-02 1.06689025e+01] [3.20555676e-02 4.99804842e-02 2.69871299e-02 3.01534095e-02
 1.01557299e-02 1.73472348e-18 9.37504552e-03 1.73472348e-18
 3.86198275e-02 1.95466656e-01]
HE2222-0026
[ 2.35724372  8.9415501   1.47798919 -0.30995242  2.4314504   3.86481491
  0.31166469  0.42097394 -0.02003214  3.46312452] [0.03275136 0.03456595 0.02290019 0.03270726 0.01892454 0.01665557
 0.01215493 0.01129375 0.02774522 0.14313381]
HE2233+0124
[ 0.81580715  7.8419602   1.6574805   0.59356604  1.79942061  1.27170874
 -0.09043877  0.04298898  0.0